In [1]:
%cd drive/MyDrive/fever/victor_martinez/

/content/drive/MyDrive/fever/victor_martinez


In [2]:
!pip install pyspark

In [3]:
# loading tokenized data
PREPROCESSED_FILE_PATH = './data/preprocessed/amz_products_small_bert_tiny_encoded_v1.parquet/'

from src.training import utils
df = utils.read_parquet_to_pandas(PREPROCESSED_FILE_PATH)

data_size = df.shape[0]
print(f"Data size: {data_size}")
distinct_categories_count = df['main_cat'].nunique()
print(f"Total distinct main categories: {distinct_categories_count}")

# Group by 'main_cat' and count
category_counts = df['main_cat'].value_counts()
print(category_counts.head(30))

Data size: 1248417
Total distinct main categories: 22
main_cat
Tools & Home Improvement     74358
Automotive                   73434
Arts, Crafts & Sewing        72556
Toys & Games                 72034
Office Products              71681
Amazon Home                  71362
Grocery                      70184
Sports & Outdoors            70177
Books                        69685
Computers                    67156
Movies & TV                  60608
Amazon Fashion               59747
Cell Phones & Accessories    59432
Pet Supplies                 57490
Industrial & Scientific      55503
All Electronics              52878
Digital Music                48319
Camera & Photo               36687
Musical Instruments          35873
Home Audio & Theater         33401
Video Games                  21736
Health & Personal Care       14116
Name: count, dtype: int64


In [18]:
from transformers import AutoTokenizer
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-tiny')

df_sampled = df.sample(frac=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(
    df_sampled[['input_ids', 'attention_mask']],
    df_sampled['main_cat'],
    test_size=0.8,  # Now 80% of the data is used for testing
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    df_sampled[['input_ids', 'attention_mask']],  # Directly use the precomputed columns
    df_sampled['main_cat'],
    test_size=0.2,
    random_state=42
)

def extract_features(df):
    """
    Extracts features from DataFrame columns containing precomputed tokenization results.

    Args:
        df (DataFrame): DataFrame containing 'input_ids' and potentially 'attention_mask'.

    Returns:
        np.ndarray: Extracted features as numpy arrays.
    """
    # Convert the list of lists (or similar structure) to a NumPy array
    input_ids = np.stack(df['input_ids'].to_numpy())
    return input_ids

# Extract features
X_train_encoded = extract_features(X_train)
X_test_encoded = extract_features(X_test)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
print(X_train_encoded.shape)
print(X_test_encoded.shape)


(199746, 512)
(49937, 512)


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, confusion_matrix

# Create and train the model

# Adjust the pipeline to include Truncated SVD
svm_clf = make_pipeline(
    StandardScaler(with_mean=False),
    TruncatedSVD(n_components=2),  # Reduce dimensions
    SVC(kernel='linear')
)

sgd_clf = make_pipeline(
    StandardScaler(with_mean=False),  # Important to set with_mean=False for sparse data
    TruncatedSVD(n_components=100),  # Optional, consider the trade-off between dimension reduction and information loss
    SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, random_state=42)
)

svm_clf.fit(X_train_encoded, y_train)
# sgd_clf.fit(X_train_encoded, y_train)






In [7]:

# Evaluate the model

y_pred = svm_clf.predict(X_test_encoded)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

accuracy = svm_clf.score(X_test_encoded, y_test)
print(f"Test accuracy: {accuracy}")

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [21]:

# Evaluate the model

y_pred = sgd_clf.predict(X_test_encoded)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

accuracy = sgd_clf.score(X_test_encoded, y_test)
print(f"Test accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                           precision    recall  f1-score   support

          All Electronics       0.00      0.00      0.00      2047
           Amazon Fashion       0.03      0.00      0.00      2387
              Amazon Home       0.08      0.10      0.09      2860
    Arts, Crafts & Sewing       0.09      0.14      0.11      2868
               Automotive       0.07      0.02      0.03      2926
                    Books       0.10      0.90      0.18      2778
           Camera & Photo       0.07      0.13      0.09      1473
Cell Phones & Accessories       0.07      0.14      0.10      2418
                Computers       0.00      0.00      0.00      2612
            Digital Music       0.07      0.02      0.03      1929
                  Grocery       0.00      0.00      0.00      2773
   Health & Personal Care       0.05      0.02      0.03       569
     Home Audio & Theater       0.03      0.06      0.04      1363
  Industrial & Scientific       0.11      0.13      0.12     